In [63]:
!pip install spacy
!pip install pandas
!pip install -U pip setuptools wheel
!python -m spacy download en_core_web_sm
!pip install openpyxl
!pip install scikit-learn
!pip install gensim

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 330.3 kB/s eta 0:00:39
     --------------------------------------- 0.0/12.8 MB 281.8 kB/s eta 0:00:46
     --------------------------------------- 0.1/12.8 MB 595.3 kB/s eta 0:00:22
      -------------------------------------- 0.2/12.8 MB 787.7 kB/s eta 0:00:17
      -------------------------------------- 0.3/12.8 MB 947.5 kB/s eta 0:00:14
     - -------------------------------------- 0.4/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.5/12.8 MB 1.5 MB/s eta 0:00:09
     -- ------------------------------------- 0.7/12.8 MB 1.7 MB/s eta 0:00:08
     -- ------------------------------------- 0.9/12.8 MB 1.9 MB/s eta 0:00:07
     --- ------------------------------------ 1.0/12.8 MB 2.0 MB/s eta 0:00:06
     --- ------------------------------------ 1.2/12.8 MB 2.1


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
import pandas as pd
import spacy
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from scipy import sparse
from sklearn.utils.extmath import randomized_svd


### **Preprocessing the text**
The steps will be: 
* Tokenization
* Lemmatization
* Cleaning the Lemmatized tokens

In [65]:
nlp = spacy.load('en_core_web_sm')

*Functions (using spaCy) for Tokenization, Lemmatization & Cleaning*

In [66]:
def tokenize_spacy(text):
    doc = nlp(text)
    return [token.text for token in doc]

def lemmatize_spacy(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc]

def clean_text_spacy(tokens):
    cleaned_tokens = [token.lower() for token in tokens if token.isalpha()]
    return cleaned_tokens

*We will be using a Whatsapp conversation as our corpus*

In [67]:
with open('_chat.txt', 'r', encoding='utf-8') as file:
    content = file.read()

data = []
for row in content.split('\n'):
    data.append(row[row.rfind(':') + 1:])

whatsapp_df = pd.DataFrame({"whatsapp_text":data})

*Applying Tokenization*

In [68]:
whatsapp_df['scraped_tokens'] = whatsapp_df['whatsapp_text'].apply(tokenize_spacy)

*Applying Lemmatization*

In [69]:
whatsapp_df['scraped_lemmatization'] = whatsapp_df['whatsapp_text'].apply(lemmatize_spacy)

*Cleaning the lemmatized tokens*

In [70]:
whatsapp_df['cleaned_tokens'] = whatsapp_df['scraped_lemmatization'].apply(clean_text_spacy)

In [71]:
# whatsapp_df_copy = whatsapp_df.copy()
# whatsapp_df_copy['scraped_tokens'] = whatsapp_df_copy['scraped_tokens'].apply(lambda x: ' '.join(x))
# whatsapp_df_copy['scraped_lemmatization'] = whatsapp_df_copy['scraped_lemmatization'].apply(lambda x: ' '.join(x))
# whatsapp_df_copy['cleaned_tokens'] = whatsapp_df_copy['cleaned_tokens'].apply(lambda x: ' '.join(x))

### **Remove Stop Words**

In [72]:
def remove_stop_words(tokens):
    stop_words = nlp.Defaults.stop_words
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

In [75]:
whatsapp_df['filtered_tokens'] = whatsapp_df['cleaned_tokens'].apply(remove_stop_words)

In [76]:
whatsapp_df_copy = whatsapp_df.copy()
whatsapp_df_copy['filtered_tokens'] = whatsapp_df_copy['filtered_tokens'].apply(lambda x: ' '.join(x))
whatsapp_df_copy.to_excel('whatsapp_df.xlsx')
whatsapp_df_copy.head()

,whatsapp_text,scraped_tokens,scraped_lemmatization,cleaned_tokens,filtered_tokens
0,‎Messages and calls are end-to-end encrypted....,"[ , ‎Messages, and, calls, are, end, -, to, -,...","[ , ‎message, and, call, be, end, -, to, -, en...","[and, call, be, end, to, end, encrypt, no, one...",end end encrypt outside chat whatsapp read listen
1,‎You created group “מדברים באנגלית”,"[ , ‎You, created, group, “, מדברים, באנגלית, ”]","[ , ‎you, create, group, "", מדברים, באנגלית, ""]","[create, group, מדברים, באנגלית]",create group מדברים באנגלית
2,hey noam how are you?,"[ , hey, noam, how, are, you, ?]","[ , hey, noam, how, be, you, ?]","[hey, noam, how, be, you]",hey noam
3,Hey!! I’m good thanks how about you,"[ , Hey, !, !, I, ’m, good, thanks, how, about...","[ , Hey, !, !, I, ’m, good, thank, how, about,...","[hey, i, good, thank, how, about, you]",hey good thank
4,What are you going to do today,"[ , What, are, you, going, to, do, today]","[ , what, be, you, go, to, do, today]","[what, be, you, go, to, do, today]",today


### **Applying Feature Extraction by the following algorithms:**
* BOW
* TF-IDF
* Word embedding by WORD2VEC

**Bow algorithm**

In [77]:
def bow_extraction(texts):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(texts)
    return X.toarray(), vectorizer.get_feature_names_out()

In [78]:
bow_features, bow_feature_names = bow_extraction(whatsapp_df_copy['filtered_tokens'])
print("BoW Features:\n", bow_features)
print("Feature Names:\n", bow_feature_names)

BoW Features:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Feature Names:
 ['actually' 'add' 'afeka' 'ai' 'amazing' 'analysis' 'anytime' 'article'
 'ask' 'awesome' 'backend' 'bad' 'balance' 'balcony' 'basil' 'beautiful'
 'believe' 'big' 'bit' 'board' 'book' 'bump' 'bunch' 'busy' 'cake' 'care'
 'catch' 'challenge' 'change' 'character' 'chat' 'check' 'coffee' 'come'
 'conversation' 'cooking' 'cool' 'couple' 'course' 'create' 'curious'
 'currently' 'day' 'debug' 'deck' 'deep' 'definitely' 'development'
 'difference' 'dive' 'drag' 'easy' 'eat' 'edit' 'emma' 'encrypt' 'end'
 'enhance' 'excited' 'exercise' 'exist' 'exit' 'fantastic' 'far'
 'fascinating' 'finally' 'finish' 'forget' 'forward' 'fresh' 'fun' 'game'
 'gang' 'garden' 'gardening' 'generate' 'good' 'got' 'gpt' 'great'
 'gripping' 'group' 'grow' 'haha' 'hand' 'hangover' 'harari' 'hear' 'herb'
 'hey' 'highly' 'history' 'hit' 'hobby' 'hope' 'humankind' 'idea'
 

**TF-IDF algorithm**

In [79]:
def tfidf_extraction(texts):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(texts)
    return X.toarray(), vectorizer.get_feature_names_out()

In [80]:
tfidf_features, tfidf_feature_names = tfidf_extraction(whatsapp_df_copy['filtered_tokens'])
print("TF-IDF Features:\n", tfidf_features)
print("Feature Names:\n", tfidf_feature_names)

TF-IDF Features:
 [[0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.5 0.5]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]
Feature Names:
 ['actually' 'add' 'afeka' 'ai' 'amazing' 'analysis' 'anytime' 'article'
 'ask' 'awesome' 'backend' 'bad' 'balance' 'balcony' 'basil' 'beautiful'
 'believe' 'big' 'bit' 'board' 'book' 'bump' 'bunch' 'busy' 'cake' 'care'
 'catch' 'challenge' 'change' 'character' 'chat' 'check' 'coffee' 'come'
 'conversation' 'cooking' 'cool' 'couple' 'course' 'create' 'curious'
 'currently' 'day' 'debug' 'deck' 'deep' 'definitely' 'development'
 'difference' 'dive' 'drag' 'easy' 'eat' 'edit' 'emma' 'encrypt' 'end'
 'enhance' 'excited' 'exercise' 'exist' 'exit' 'fantastic' 'far'
 'fascinating' 'finally' 'finish' 'forget' 'forward' 'fresh' 'fun' 'game'
 'gang' 'garden' 'gardening' 'generate' 'good' 'got' 'gpt' 'great'
 'gripping' 'group' 'grow' 'haha' 'hand' 'hangover' 'harari' 'hear'

**Word Embeddings by Word2Vec**

In [81]:
def word2vec_extraction(tokens_list, vector_size=100, window=5, min_count=1, workers=4):
    model = Word2Vec(sentences=tokens_list, vector_size=vector_size, window=window, min_count=min_count, workers=workers)
    return model

In [82]:
tokenized_texts = [text.split() for text in whatsapp_df_copy['filtered_tokens']]

word2vec_model = word2vec_extraction(tokenized_texts)
similar_words = word2vec_model.wv.most_similar('finish')
print(similar_words)

[('nice', 0.27789372205734253), ('anytime', 0.24751438200473785), ('exercise', 0.2450217306613922), ('upgrade', 0.2410586029291153), ('things', 0.23789504170417786), ('busy', 0.23460040986537933), ('star', 0.22931070625782013), ('backend', 0.2289053350687027), ('excited', 0.2271011471748352), ('little', 0.19759514927864075)]


## **Glove Explanation**

GloVe (Global Vectors for Word Representation) is an unsupervised learning algorithm for obtaining vector representations (embeddings) for words. These embeddings capture semantic relationships between words based on their co-occurrence statistics in a large corpus of text. Unlike Word2Vec, which learns embeddings by predicting context words given a target word (skip-gram model) or predicting a target word given context words (continuous bag of words model), GloVe learns embeddings by factorizing the word co-occurrence matrix.

Explanation of GloVe:
* Co-occurrence Matrix: GloVe starts with a word-word co-occurrence matrix where each element 
𝑋𝑖𝑗 ​represents how often word 𝑗 appears in the context of word 𝑖 within a fixed window size.

* Objective: The goal of GloVe is to learn word embeddings such that the dot product of two word vectors corresponds to the logarithm of their co-occurrence probability.

* Training: GloVe uses stochastic gradient descent to minimize a loss function that measures the discrepancy between the dot product of word vectors and the logarithm of their co-occurrence probabilities.

* Advantages: GloVe embeddings tend to capture global semantic meanings better than some other models, and they often perform well in tasks requiring understanding of word relationships and analogies.

In [83]:
import numpy as np
from scipy.sparse import lil_matrix

def build_cooccurrence_matrix(corpus, window_size=2):
    vocab = list(set(corpus))
    word_to_id = {word: i for i, word in enumerate(vocab)}
    cooccurrence = lil_matrix((len(vocab), len(vocab)), dtype=np.float64)
    
    for i, word in enumerate(corpus):
        left_context = max(0, i - window_size)
        right_context = min(len(corpus), i + window_size + 1)
        for j in range(left_context, right_context):
            if i != j:
                cooccurrence[word_to_id[word], word_to_id[corpus[j]]] += 1
    
    return cooccurrence.tocsr(), word_to_id

def glove_loss(X, W, U, b, c):
    diff = W.dot(U.T) + b[:, np.newaxis] + c[np.newaxis, :] - np.log(X.toarray() + 1)
    squared_error = np.sum(diff ** 2)
    return squared_error

def train_glove(X, vector_size=50, iterations=50, learning_rate=0.01, clip_value=1.0):
    vocab_size = X.shape[0]
    W = np.random.randn(vocab_size, vector_size) / np.sqrt(vector_size)
    U = np.random.randn(vocab_size, vector_size) / np.sqrt(vector_size)
    b = np.zeros(vocab_size)
    c = np.zeros(vocab_size)
    
    for iteration in range(iterations):
        error = glove_loss(X, W, U, b, c)
        if iteration % 10 == 0:
            print(f"Iteration {iteration}: Loss = {error}")
        
        # Compute gradients
        diff = W.dot(U.T) + b[:, np.newaxis] + c[np.newaxis, :] - np.log(X.toarray() + 1)
        grad_W = 2 * diff.dot(U)
        grad_U = 2 * diff.T.dot(W)
        grad_b = 2 * np.sum(diff, axis=1)
        grad_c = 2 * np.sum(diff, axis=0)
        
        # Gradient clipping
        grad_W = np.clip(grad_W, -clip_value, clip_value)
        grad_U = np.clip(grad_U, -clip_value, clip_value)
        grad_b = np.clip(grad_b, -clip_value, clip_value)
        grad_c = np.clip(grad_c, -clip_value, clip_value)
        
        # Update parameters
        W -= learning_rate * grad_W
        U -= learning_rate * grad_U
        b -= learning_rate * grad_b
        c -= learning_rate * grad_c
    
    return (W + U) / 2

In [84]:
corpus = []
for sentence in whatsapp_df_copy['filtered_tokens']:
    for word in sentence.split(' '):
        corpus.append(word)

In [85]:
X, word_to_id = build_cooccurrence_matrix(corpus)
word_vectors = train_glove(X, vector_size=5, iterations=100, learning_rate=0.01, clip_value=5.0)

Iteration 0: Loss = 9547.968105155907
Iteration 10: Loss = 886.8382972979341
Iteration 20: Loss = 770.1583807250861
Iteration 30: Loss = 753.4867345861778
Iteration 40: Loss = 736.8506720763112
Iteration 50: Loss = 722.9037751401183
Iteration 60: Loss = 712.6764474995944
Iteration 70: Loss = 705.9745987429415
Iteration 80: Loss = 701.477432781958
Iteration 90: Loss = 698.0580476327473


In [86]:
for word, idx in word_to_id.items():
    print(f"{word}: {word_vectors[idx]}")

: [-0.55860724 -0.30560315 -0.08152729 -0.27706367 -0.05147736]
challenge: [ 0.03521697  0.02115465 -0.03628242 -0.12011233 -0.0378793 ]
rewarding: [0.07359107 0.10351958 0.03730213 0.07946659 0.01301849]
stranger: [ 0.06430299  0.09680638  0.07266604 -0.0337872   0.15046244]
difference: [ 0.04797072 -0.1418214   0.16422065  0.08450146  0.04560523]
end: [ 0.12435624  0.00549067 -0.04257162 -0.1324257   0.01576025]
debug: [-0.00820362  0.09662148  0.06509091  0.04796182  0.03761253]
exist: [-0.02772332  0.07703624 -0.00084299 -0.07894938 -0.0466522 ]
cake: [-0.00324985  0.00208952  0.1046634  -0.07894299 -0.07672568]
school: [ 0.10771834  0.00160645 -0.07924419 -0.02076776 -0.18884321]
travel: [ 0.01400674 -0.10255247 -0.1624058   0.12760566  0.14677435]
forward: [-0.0606057  -0.30206211 -0.053011    0.30006419  0.31107129]
requirement: [ 0.00786203 -0.04486748 -0.03929018  0.05008567  0.07031282]
beautiful: [-0.05965714 -0.03511297 -0.02798982 -0.11825527 -0.03675119]
balance: [-0.0149

### **Explanation of applying GloVe:**
The GloVe vectors represent semantic embeddings of the provided words. Each vector captures the meaning and context of the word in a high-dimensional space. 

Analysis based on the vectors:

Challenge: This word has a slightly positive direction in the second dimension, suggesting a mildly positive connotation or context.  
Rewarding: The vector indicates a positive sentiment, possibly related to satisfaction or positive outcomes.  
Stranger: This vector shows a mix of directions, indicating complexity or varied contexts associated with the word.  
Difference: The vector suggests a directional contrast, possibly indicating variation or distinction.  
End: The vector shows positive directions in various dimensions, indicating completeness or finality.  
Debug: This vector suggests a technical context, with directions pointing towards problem-solving or software debugging.  
Exist: The vector shows a mixture, possibly indicating existence or being present.  
Travel: This vector suggests movement or direction related to travel.  
School: The vector points towards education or academic contexts.  
Forward: This vector indicates directionality or progression.  

Each word's vector encapsulates its semantic nuances and associations, useful for tasks like word similarity, sentiment analysis, or understanding context in natural language processing applications.

In [87]:
# # sharon's code
# def build_cooccurrence_matrix(corpus, window_size=2):
#     vocab = list(set(corpus))
#     word_to_id = {word: i for i, word in enumerate(vocab)}
#     cooccurrence = sparse.lil_matrix((len(vocab), len(vocab)), dtype=np.float64)
    
#     for i, word in enumerate(corpus):
#         left_context = max(0, i - window_size)
#         right_context = min(len(corpus), i + window_size + 1)
#         for j in range(left_context, right_context):
#             if i != j:
#                 cooccurrence[word_to_id[word], word_to_id[corpus[j]]] += 1
    
#     return cooccurrence.tocsr(), word_to_id

# def glove_loss(X, W, b, U, c):
#     return np.sum(np.power(W.dot(U.T) + b[:, np.newaxis] + c[np.newaxis, :] - np.log(X.toarray() + 1), 2))

# def train_glove(X, vector_size=50, iterations=50, learning_rate=0.05):
#     vocab_size = X.shape[0]
#     W = np.random.randn(vocab_size, vector_size) / np.sqrt(vector_size)
#     U = np.random.randn(vocab_size, vector_size) / np.sqrt(vector_size)
#     b = np.zeros(vocab_size)
#     c = np.zeros(vocab_size)
    
#     for _ in range(iterations):
#         grad_W = 2 * (W.dot(U.T) + b[:, np.newaxis] + c[np.newaxis, :] - np.log(X.toarray() + 1)).dot(U)
#         grad_U = 2 * (W.dot(U.T) + b[:, np.newaxis] + c[np.newaxis, :] - np.log(X.toarray() + 1)).T.dot(W)
#         grad_b = 2 * np.sum(W.dot(U.T) + b[:, np.newaxis] + c[np.newaxis, :] - np.log(X.toarray() + 1), axis=1)
#         grad_c = 2 * np.sum(W.dot(U.T) + b[:, np.newaxis] + c[np.newaxis, :] - np.log(X.toarray() + 1), axis=0)
        
#         W -= learning_rate * grad_W
#         U -= learning_rate * grad_U
#         b -= learning_rate * grad_b
#         c -= learning_rate * grad_c
        
#         if _ % 10 == 0:
#             print(f"Iteration {_}, Loss: {glove_loss(X, W, b, U, c)}")
    
#     return (W + U) / 2

In [88]:
# corpus = []

# for sentence in whatsapp_df_copy['filtered_tokens']:
#     for word in sentence.split(' '):
#         corpus.append(word)

In [89]:
# X, word_to_id = build_cooccurrence_matrix(corpus)
# word_vectors = train_glove(X, vector_size=5, iterations=100)

# for word, idx in word_to_id.items():
#     print(f"{word}: {word_vectors[idx]}")

### **Applying tagging by CYK**

We are asked to show apply the CYK tagging to 5 sentences, 1 manually and 4 with libraries and built-in functions.

In [121]:
def cyk_parse(sentence, grammar):
    # Step 1: Tokenization
    tokens = sentence.split()
    n = len(tokens)
    table = [[set() for _ in range(n+1)] for _ in range(n+1)]
	
    # Step 2: Initialization
    for i in range(1, n+1):
        for rule in grammar:
            if rule[1] == tokens[i-1]:
                table[i][i].add(rule[0])

    # Step 3: Rule Application
    for length in range(2, n+1):
        for i in range(1, n-length+2):
            j = i + length - 1
            for k in range(i, j):
                for rule in grammar:
                    if len(rule) == 3:
                        for left in table[i][k]:
                            for right in table[k+1][j]:
                                if rule[1] in left and rule[2] in right:
                                    table[i][j].add(rule[0])

    # Step 4: Backtracking
    if 'S' in table[1][n]:
        return True, table
    else:
        return False, table

In [122]:
# Creating the Grammar & table manually
sentence = whatsapp_df_copy['whatsapp_text'][19]

grammar = [
    ('S', 'NP', 'VP'),
    ('NP', 'DET', 'NP'),
    ('NP', 'adjective', 'Noun'),
    ('VP', 'Verb', 'NP'),
    ('Noun', 'today'),
    ('Noun', 'day'),
    ('Verb', 'was'),
    ('Det', 'a'),
    ('adjective', 'beautiful')
]

# Call the CYK parser
parsed, table = cyk_parse(sentence, grammar)

# Print the parse table and whether the sentence was parsed or not
if parsed:
    print("Input sentence: ", sentence)
    print("Parse table: ")
    for row in table:
        print(row)
else:
    print("Input sentence: ", sentence)
    for row in table:
        print(row)
    print("Sentence not parsed.")

[[set(), set(), set(), set(), set(), set()], [set(), {'Noun'}, set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()]]
[[set(), set(), set(), set(), set(), set()], [set(), {'Noun'}, set(), set(), set(), set()], [set(), set(), {'Verb'}, set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()]]
[[set(), set(), set(), set(), set(), set()], [set(), {'Noun'}, set(), set(), set(), set()], [set(), set(), {'Verb'}, set(), set(), set()], [set(), set(), set(), {'Det'}, set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()]]
[[set(), set(), set(), set(), set(), set()], [set(), {'Noun'}, set(), set(), set(), set()], [set(), set(), {'Verb'}, set(), set(), set()], [set(), set(), set(), {'Det'}, set(), set()], [se

In [117]:
# tokenizing the sentence
tokens = ["today", "was", "a", "beautiful", "day"]

In [106]:
# initializing the cyk table
n = len(sentence)
cyk_table = [[[] for _ in range(n)] for _ in range(n)]

for j in range(n):
    word = sentence[j]
    for rule in grammar:
        if len(rule) == 2 and rule[1] == word:
            cyk_table[j][j].append(rule[0])

# Fill the CYK table for lengths > 1
for length in range(2, n + 1):
    for i in range(n - length + 1):
        j = i + length - 1
        for k in range(i, j):
            for rule in grammar:
                if len(rule) == 3:
                    lhs, rhs1, rhs2 = rule
                    if rhs1 in cyk_table[i][k] and rhs2 in cyk_table[k + 1][j]:
                        cyk_table[i][j].append(lhs)

# Print the CYK table
for row in cyk_table:
    print(row)

[['Noun'], [], [], [], []]
[[], ['Verb'], [], [], []]
[[], [], ['Det'], [], []]
[[], [], [], ['adjective'], ['NP']]
[[], [], [], [], ['Noun']]


In [111]:
def cyk_parse(grammar, sentence):
    # Length of the sentence
    n = len(sentence)

    # Initialize the CYK table
    cyk_table = [[set() for _ in range(n)] for _ in range(n)]

    # Fill the table for length 1 substrings (single words)
    for j in range(n):
        word = sentence[j]
        for rule in grammar:
            if len(rule) == 2 and rule[1] == word:
                cyk_table[j][j].add(rule[0])

    # Fill the table for substrings of length 2 to n
    for length in range(2, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            for k in range(i, j):
                for rule in grammar:
                    if len(rule) == 3:
                        A, B, C = rule
                        if B in cyk_table[i][k] and C in cyk_table[k + 1][j]:
                            cyk_table[i][j].add(A)

    # Print the CYK table
    print("CYK Table:")
    for row in cyk_table:
        print(["{" + ", ".join(cell) + "}" for cell in row])

    # Check if the start symbol 'S' is in the top-right cell of the table
    if 'S' in cyk_table[0][n - 1]:
        print("\nThe sentence can be derived from the start symbol 'S'.")
    else:
        print("\nThe sentence cannot be derived from the start symbol 'S'.")

# Define the grammar rules
grammar = [
    ('S', 'NP', 'VP'),
    ('NP', 'Noun'),
    ('NP', 'Det', 'NP'),
    ('NP', 'adjective', 'Noun'),
    ('VP', 'Verb', 'NP'),
    ('Noun', 'today'),
    ('Noun', 'day'),
    ('Verb', 'was'),
    ('Det', 'a'),
    ('adjective', 'beautiful')
]

# Define the sentence
sentence = ["today", "was", "a", "beautiful", "day"]

# Run the CYK parser
cyk_parse(grammar, sentence)


CYK Table:
['{Noun}', '{}', '{}', '{}', '{}']
['{}', '{Verb}', '{}', '{}', '{VP}']
['{}', '{}', '{Det}', '{}', '{NP}']
['{}', '{}', '{}', '{adjective}', '{NP}']
['{}', '{}', '{}', '{}', '{Noun}']

The sentence cannot be derived from the start symbol 'S'.


In [ ]:


parsed, table = cyk_parse(sentence, grammar)

# Print the parse table and whether the sentence was parsed or not
if parsed:
    print("Input sentence: ", sentence)
    print("Parse table: ")
    for row in table:
        print(row)
else:
    print("Input sentence: ", sentence)
    print("Sentence not parsed.")

In [123]:
# Define the context-free grammar in CNF
grammar = [
    ('S', 'NP', 'VP'),
    ('NP', 'Det', 'Noun'),
    ('VP', 'Verb', 'NP'),
    ('Det', 'the'),
    ('Det', 'a'),
    ('Noun', 'cat'),
    ('Noun', 'dog'),
    ('Verb', 'chased'),
    ('Verb', 'ate')
]

# Input sentence to be parsed
sentence = "the cat chased a dog"

# Call the CYK parser
parsed, table = cyk_parse(sentence, grammar)

# Print the parse table and whether the sentence was parsed or not
if parsed:
    print("Input sentence: ", sentence)
    print("Parse table: ")
    for row in table:
        print(row)
else:
    print("Input sentence: ", sentence)
    print("Sentence not parsed.")

[[set(), set(), set(), set(), set(), set()], [set(), {'Det'}, set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()]]
[[set(), set(), set(), set(), set(), set()], [set(), {'Det'}, set(), set(), set(), set()], [set(), set(), {'Noun'}, set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()]]
[[set(), set(), set(), set(), set(), set()], [set(), {'Det'}, set(), set(), set(), set()], [set(), set(), {'Noun'}, set(), set(), set()], [set(), set(), set(), {'Verb'}, set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()]]
[[set(), set(), set(), set(), set(), set()], [set(), {'Det'}, set(), set(), set(), set()], [set(), set(), {'Noun'}, set(), set(), set()], [set(), set(), set(), {'Verb'}, set(), set()], [set(